In [1]:
!tree ../dataset

../dataset
├── en_espanol
│   ├── docx2txt.py
│   ├── Второй_жанр_исходная.txt
│   └── Первый_жанр_исходная.txt
├── Второй_жанр_исходная.txt
├── Первый_жанр_исходная.txt
├── Сокращение по частям речи
│   ├── 1.Первый жанр исходная выборка.txt
│   ├── 2.Первый жанр без клауз, включающих наречия.txt
│   ├── 3.Первый жанр без клауз, включающих глаголы.txt
│   ├── 4.Первый жанр без клауз, включающих глаголы и наречия.txt
│   ├── 5.без клауз, включающих существительные.txt
│   ├── 6.без клауз, включающих местоимения.txt
│   ├── 7.без клауз, включающих существительные и прилагательные.txt
│   ├── 8.без слов функциональных.txt
│   ├── Без прилагательных второй жанр.txt
│   ├── Без прилагательных первый жанр.txt
│   ├── Случайные выборки.txt
│   ├── без клауз, включающих местоимения.txt
│   └── без слов функциональных.txt
└── сокращение по частотности
    ├── 1а_ без сокращений.txt
    ├── 1б_Изъяты лексемы с частотой выше 100.txt
    ├── 1в_Изъяты лексемы с частотой выше 49.txt
    ├── 1г_Изъ

In [4]:
import stanza
import os
from pathlib import Path

# Configurar Stanza para ruso
stanza.download('ru')
nlp = stanza.Pipeline('ru', processors='tokenize,pos,lemma')

# Directorios
input_dir = '.'  # Directorio con Первый_жанр_исходная.txt y Второй_жанр_исходная.txt
output_dir = '../dataset/Сокращение по частям речи'
os.makedirs(output_dir, exist_ok=True)

# Archivos de entrada
input_files = {
    'Первый_жанр': '../dataset/Первый_жанр_исходная.txt',
    'Второй_жанр': '../dataset/Второй_жанр_исходная.txt'
}

# Configuración de modificaciones
modifications = [
#     {
#         'name': 'без клауз, включающих существительные',
#         'prefix': {'Первый_жанр': '5.', 'Второй_жанр': 'Второй_жанр Без существительных'},
#         'pos_to_remove': ['NOUN'],
#         'remove_clause': True
#     },
    {
        'name': 'без клауз, включающих местоимения',#funciona
        'prefix': {'Первый_жанр': '5.', 'Второй_жанр': 'Второй_жанр '},
        'pos_to_remove': ['PRON'],
        'remove_clause': True
    },
#     {
#         'name': 'без клауз, включающих существительные и прилагательные',
#         'prefix': {'Первый_жанр': '7.', 'Второй_жанр': 'Второй_жанр Без существительных и прилагательных'},
#         'pos_to_remove': ['NOUN', 'ADJ'],
#         'remove_clause': True
#     },
    {
        'name': 'без слов функциональных',#funciona
        'prefix': {'Первый_жанр': '6.', 'Второй_жанр': 'Второй_жанр '},
        'pos_to_remove': ['PART', 'ADP', 'CCONJ'],
        'remove_clause': False
    }
]

def process_text(text, modification):
    doc = nlp(text)
    output_sentences = []
    pos_to_remove = modification['pos_to_remove']
    remove_clause = modification['remove_clause']
    
    for sent in doc.sentences:
        if remove_clause:
            # Verificar si la oración contiene alguna palabra con las POS a eliminar
            has_pos = any(word.upos in pos_to_remove for word in sent.words)
            if not has_pos:
                output_sentences.append(sent.text)
        else:
            # Eliminar palabras individuales con las POS especificadas
            filtered_words = [word.text for word in sent.words if word.upos not in pos_to_remove]
            if filtered_words:  # Solo añadir si hay palabras restantes
                output_sentences.append(' '.join(filtered_words))
    
    return '\n'.join(output_sentences)

# Procesar cada archivo
for genre, input_file in input_files.items():
    input_path = Path(input_dir) / input_file
    if not input_path.exists():
        print(f"Archivo no encontrado: {input_path}")
        continue
    
    with open(input_path, 'r', encoding='utf-8') as f:
        text = f.read()
    
    for mod in modifications:
        # Generar texto modificado
        modified_text = process_text(text, mod)
        if not modified_text.strip():
            print(f"Advertencia: El texto procesado para {mod['name']} en {genre} está vacío")
        
        # Crear nombre del archivo de salida
        prefix = mod['prefix'][genre]
        output_filename = f"{prefix}{mod['name']}.txt"
        output_path = Path(output_dir) / output_filename
        
        # Guardar resultado
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(modified_text)
        print(f"Generado: {output_path}")

print("Procesamiento completado.")

2025-04-22 18:33:07 INFO: Downloaded file to /root/stanza_resources/resources.json
2025-04-22 18:33:07 INFO: Downloading default packages for language: ru (Russian) ...
2025-04-22 18:33:08 INFO: File exists: /root/stanza_resources/ru/default.zip
2025-04-22 18:33:11 INFO: Finished downloading models and saved to /root/stanza_resources
2025-04-22 18:33:11 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-04-22 18:33:11 INFO: Downloaded file to /root/stanza_resources/resources.json
2025-04-22 18:33:11 INFO: Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| pos       | syntagrus_charlm   |
| lemma     | syntagrus_nocharlm |

2025-04-22 18:33:11 INFO: Using device: cuda
2025-04-22 18:33:11 INFO: Loading: tokenize
2025-04-22 18:33:11 INFO: L

Generado: ../dataset/Сокращение по частям речи/5.без клауз, включающих местоимения.txt
Generado: ../dataset/Сокращение по частям речи/6.без слов функциональных.txt
Generado: ../dataset/Сокращение по частям речи/Второй_жанр без клауз, включающих местоимения.txt
Generado: ../dataset/Сокращение по частям речи/Второй_жанр без слов функциональных.txt
Procesamiento completado.
